# Get an authenticated AzureOpenAI client
Your Azure AI Foundry project may have one or more OpenAI models deployed that support chat completions. Use the code below to get an already authenticated AzureOpenAI from the openai package, and execute a chat completions call.<br/><br/>
**IMPORTANT**: THE INTERACTIVE USER MUST HAVE THE ```Cognitive Services Contributor``` ROLE ON THE AZURE AI SERVICES SERVICE.
<br/><br/>
First, install the package:
```
    pip install openai
```
Then run the code below. Replace **gpt-4o** with your model deployment name, and update the api_version value with one found in the "Data plane - inference" row in this table.

# Constants and Libraries

## Only if needed
```
import os

# Login with tenant ID
os.system("az login --tenant 3ad0b905-34ab-4116-93d9-c1dcc2d35af6 --output none")

# Set the subscription programmatically
os.system("az account set --subscription eca2eddb-0f0c-4351-a634-52751499eeea")
```

In [1]:
import os
from IPython.display import Markdown, display
from dotenv import load_dotenv # requires python-dotenv
from PIL import Image # requires pip install pillow
from datetime import datetime
from common.agents_helper_functions_NEW import *
import importlib.metadata
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

if not load_dotenv("./../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
else:
    print("Environment variables have been loaded ;-)")


project_endpoint = os.environ["AIF_STD_PROJECT_ENDPOINT"]
deployment_name =  os.environ["MODEL_DEPLOYMENT_NAME"]
api_version = "2025-02-01-preview" # os.environ["OPENAI_API_VERSION"] # >="2025-03-01-preview" for Responses APIs otherwise <="2024-01-01-preview" fails

print(f'Project Endpoint: <{project_endpoint}>')
print(f'azure-ai-projects library installed version: {importlib.metadata.version("azure-ai-projects")}')
print(f'azure-ai-agents library installed version: {importlib.metadata.version("azure-ai-agents")}')
print(f"api_version = {api_version}")

Environment variables have been loaded ;-)
Project Endpoint: <https://aif2stdsvhdu2.services.ai.azure.com/api/projects/aif2stdwusprj01hdu2>
azure-ai-projects library installed version: 1.0.0
azure-ai-agents library installed version: 1.1.0
api_version = 2025-02-01-preview


# Create AI Foundry Project Client

In [2]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

project_client

# Extract and Cast the OpenAI Client from AI Foundry Project Client

In [3]:
from openai import AzureOpenAI

aoai_client: AzureOpenAI = project_client.get_openai_client(api_version=api_version)

print(f"type(aoai_client): {type(aoai_client)}")

response = aoai_client.chat.completions.create(
    model=deployment_name,
    messages=[
        {
            "role": "user",
            "content": "How many feet are in a mile?",
        },
    ],
)

print(response.choices[0].message.content)

type(aoai_client): <class 'openai.lib.azure.AzureOpenAI'>
There are 5,280 feet in a mile.


## The following cells works only if `api_version >= "2024-07-01-preview"`

In [4]:
response = aoai_client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": "Write a one-sentence bedtime story about a unicorn."
      }
  ]
)
response.output_text

BadRequestError: Error code: 400 - {'error': {'code': 'BadRequest', 'message': 'Azure OpenAI Responses API is enabled only for api-version 2025-03-01-preview and later'}}

# HIC SUNT LEONES